In [22]:
from IPython.display import display
import math
import piplite
await piplite.install("bqplot")
await piplite.install("ipywidgets")
import bqplot
from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root, fsolve

In [23]:
#Variables

n = 2 #Hill coefficient
Km = 40 #monomeres per cell
beta = 0.2
alpha0 = 0.2164 
alpha = 216.404
tau_mRNA = 2
tau_prot = 10
kd_m = math.log(2)/tau_mRNA
kd_p = math.log(2)/tau_prot
k_tl = 6.93

CmtetR0 = 1
Cmlambda0 = 4
CmlacI0 = 1
CptetR0 = 2
Cplambda0 = 1
CplacI0 = 1

In [24]:
#Calculating data for plots
    
def repressilator_model(t, data, alpha0, alpha, kd_m, kd_p, k_tl, n):
    m = data[:3]
    p = data[3:]
    dm0 = -kd_m*m[0] + alpha*(Km**n)/((Km**n)+(p[2])**n) + alpha0
    dm1 = -kd_m*m[1] + alpha*(Km**n)/((Km**n)+(p[0])**n) + alpha0
    dm2 = -kd_m*m[2] + alpha*(Km**n)/((Km**n)+(p[1])**n) + alpha0
    dp0 = -kd_p*p[0] + k_tl*m[0]
    dp1 = -kd_p*p[1] + k_tl*m[1]
    dp2 = -kd_p*p[2] + k_tl*m[2]
    return [dm0,dm1,dm2,dp0,dp1,dp2]
    
t0 = 0.0
tf = 1000.0
time = np.linspace(0.0,1000.0,10000)
initval = np.array([CmtetR0,Cmlambda0,CmlacI0,CptetR0,Cplambda0,CplacI0])
res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(alpha0,alpha,kd_m,kd_p,k_tl,n))
res = dict(res)
time = res['t']
y = res['y']

ylacI = y[5]
ytetR = y[3]
ylambda = y[4]

fig = plt.figure(title="Repressilator model", legend_location="top-left")
fig.layout.height = '500px'
fig.layout.width = '650px'

line_chart = plt.plot(x=time, y=[ylacI, ytetR, ylambda],
                     labels=["LacI","TetR", "cI"], colors=['blue', 'green', 'red'],
                     display_legend=True)

plt.xlabel("Time, min")
plt.ylabel("Number of proteins")

In [25]:
#Callback
def update_plot(tM, tP, k_tl, a, a0, CmlacI0, CmtetR0, CmcI0, CplacI0, CptetR0, CpcI0, n):
    kd_m = math.log(2)/tM
    kd_p = math.log(2)/tP
    t0 = 0.0
    tf = 1000.0
    time = np.linspace(0.0,1000.0,10000)
    initval = np.array([CmtetR0,CmcI0,CmlacI0,CptetR0,CpcI0,CplacI0])
    res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(a0,a,kd_m,kd_p,k_tl,n))
    res = dict(res)
    time = res['t']
    y = res['y']
    ylacI = y[5]
    ytetR = y[3]
    ylambda = y[4]
    line_chart.y = [ylacI, ytetR, ylambda]

In [26]:
#widgets
wdj = widgets.interactive(update_plot, tM = widgets.FloatText(value=tau_mRNA, description="tau_mRNA:", disabled=False),
tP = widgets.FloatText(value=tau_prot, description="tau_prot:", disabled=False),
k_tl = widgets.FloatText(value=k_tl, description="k_tl:", disabled=False),                          
a = widgets.FloatText(value=alpha, description="alpha:", disabled=False),
a0 = widgets.FloatText(value=alpha0, description="alpha0:", disabled=False),
CmlacI0 = widgets.FloatText(value=CmlacI0, description="CmlacI0:", disabled=False),
CmtetR0 = widgets.FloatText(value=CmtetR0, description="CmtetR0:", disabled=False),
CmcI0 = widgets.FloatText(value=Cmlambda0, description="Cmlambda0:", disabled=False),
CplacI0 = widgets.FloatText(value=CplacI0, description="CplacI0:", disabled=False),
CptetR0 = widgets.FloatText(value=CptetR0, description="CptetR0:", disabled=False),
CpcI0 = widgets.FloatText(value=Cplambda0, description="Cplambda0:", disabled=False),
n = widgets.FloatSlider(value=n, min=1.0, max=4.0, step=0.1, description="n:", disabled=False))

In [27]:
display(widgets.HBox([fig, wdj]))

<h2>Finding the fixed points</h2>

In [28]:
#Finding fixed points
def eq_points_equations(X, alpha, alpha0, kd_m, kd_p, k_tl, Km, n):
    Xm1, Xm2, Xm3, Xp1, Xp2, Xp3 = X
    f = [
        Xm1 - (alpha*(Km**n)/(kd_m*(Km**n+Xp3**n)) + alpha0/kd_m),
        Xm2 - (alpha*(Km**n)/(kd_m*(Km**n+Xp1**n)) + alpha0/kd_m),
        Xm3 - (alpha*(Km**n)/(kd_m*(Km**n+Xp2**n)) + alpha0/kd_m),
        Xp1 - k_tl*Xm1/kd_p,
        Xp2 - k_tl*Xm2/kd_p,
        Xp3 - k_tl*Xm3/kd_p
    ]
    return f

points = root(eq_points_equations, [CmtetR0, Cmlambda0, CmlacI0, CptetR0, Cplambda0, CplacI0], args=(alpha, alpha0, kd_m, kd_p, k_tl, Km, n))
#points

In [29]:
Xm1, Xm2, Xm3, Xp1, Xp2, Xp3 = points.x
Xm1, Xm2, Xm3, Xp1, Xp2, Xp3

(4.848065354130793,
 4.848065354150609,
 4.848065354168917,
 484.7035932106894,
 484.70359321267057,
 484.703593214501)

As we can see above there is exactly 1 fixed point in this system. Which type is it?

<h2>Phase portraits</h2>

In [30]:
#cI-lacI phase portrait
fig1 = plt.figure(title="Phase portrait 1", legend_location="top-left")
phase1 = plt.plot(y[4], y[5])
plt.ylabel("lacI, number of proteins")
plt.xlabel("cl, number of proteins")
fig1.layout.height = '500px'
fig1.layout.width = '500px'

In [31]:
def update_phase1(tM, tP, n):
    kd_m = math.log(2)/tM
    kd_p = math.log(2)/tP
    a = 216.404
    a0 = 0.2164
    k_tl = 6.93
    CmtetR0 = 1
    CmcI0 = 4
    CmlacI0 = 1
    CptetR0 = 2
    CpcI0 = 1
    CplacI0 = 1
    t0 = 0.0
    tf = 1000.0
    time = np.linspace(0.0,1000.0,10000)
    initval = np.array([CmtetR0,CmcI0,CmlacI0,CptetR0,CpcI0,CplacI0])
    res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(a0,a,kd_m,kd_p,k_tl, n))
    res = dict(res)
    y = res['y']
    ylacI = y[5]
    ylambda = y[4]
    phase1.x = ylambda
    phase1.y = ylacI

In [32]:
wdj1 = widgets.interactive(update_phase1, tM = widgets.FloatText(value=tau_mRNA, description="tau_mRNA:", disabled=False),
tP = widgets.FloatText(value=tau_prot, description="tau_prot:", disabled=False),
n = widgets.FloatSlider(value=n, min=1.0, max=4.0, step=0.1, description="n:", disabled=False))

In [33]:
display(widgets.HBox([fig1, wdj1]))

In [34]:
#lacI-tetR phase portrait
fig2 = plt.figure(title="Phase portrait 2", legend_location="top-left")
phase2 = plt.plot(y[5], y[3])
plt.ylabel("tetR, number of proteins")
plt.xlabel("lacI, number of proteins")
fig2.layout.height = '500px'
fig2.layout.width = '500px'

In [35]:
def update_phase2(tM, tP, n):
    kd_m = math.log(2)/tM
    kd_p = math.log(2)/tP
    a = 216.404
    a0 = 0.2164
    k_tl = 6.93
    CmtetR0 = 1
    CmcI0 = 4
    CmlacI0 = 1
    CptetR0 = 2
    CpcI0 = 1
    CplacI0 = 1
    t0 = 0.0
    tf = 1000.0
    time = np.linspace(0.0,1000.0,10000)
    initval = np.array([CmtetR0,CmcI0,CmlacI0,CptetR0,CpcI0,CplacI0])
    res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(a0,a,kd_m,kd_p,k_tl, n))
    res = dict(res)
    y = res['y']
    ylacI = y[5]
    ytetR = y[3]
    phase2.x = ylacI
    phase2.y = ytetR

In [36]:
wdj2 = widgets.interactive(update_phase2, tM = widgets.FloatText(value=tau_mRNA, description="tau_mRNA:", disabled=False),
tP = widgets.FloatText(value=tau_prot, description="tau_prot:", disabled=False),
n = widgets.FloatSlider(value=n, min=1.0, max=4.0, step=0.1, description="n:", disabled=False))

In [37]:
display(widgets.HBox([fig2, wdj2]))

In [38]:
#tetR-cI phase portrait
fig3 = plt.figure(title="Phase portrait 3", legend_location="top-left")
phase3 = plt.plot(y[3],y[4])
plt.xlabel("tetR, number of proteins")
plt.ylabel("cl, number of proteins")
fig3.layout.height = '500px'
fig3.layout.width = '500px'

In [39]:
def update_phase3(tM, tP, n):
    kd_m = math.log(2)/tM
    kd_p = math.log(2)/tP
    a = 216.404
    a0 = 0.2164
    k_tl = 6.93
    CmtetR0 = 1
    CmcI0 = 4
    CmlacI0 = 1
    CptetR0 = 2
    CpcI0 = 1
    CplacI0 = 1
    t0 = 0.0
    tf = 1000.0
    time = np.linspace(0.0,1000.0,10000)
    initval = np.array([CmtetR0,CmcI0,CmlacI0,CptetR0,CpcI0,CplacI0])
    res = solve_ivp(repressilator_model, (t0,tf), initval, method='RK45', t_eval=time, args=(a0,a,kd_m,kd_p,k_tl, n))
    res = dict(res)
    y = res['y']
    ylambda = y[4]
    ytetR = y[3]
    phase3.x = ytetR
    phase3.y = ylambda

In [40]:
wdj3 = widgets.interactive(update_phase3, tM = widgets.FloatText(value=tau_mRNA, description="tau_mRNA:", disabled=False),
tP = widgets.FloatText(value=tau_prot, description="tau_prot:", disabled=False),
n = widgets.FloatSlider(value=n, min=1.0, max=4.0, step=0.1, description="n:", disabled=False))

In [41]:
display(widgets.HBox([fig3, wdj3]))